In [1]:
import pandas as pd 
import requests
from pandas import json_normalize

1. THE THREE CELLS BELOW ARE API REQUESTS

In [2]:
response_API = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=0')
print(response_API.status_code)

200


In [3]:
response_API2 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=10000')
print(response_API2.status_code)

200


In [4]:
response_API3 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=20000')
print(response_API3.status_code)

200


In [5]:
data = response_API.json()
print(type(data))


<class 'dict'>


In [6]:
data2 = response_API2.json()
print(type(data2))

<class 'dict'>


In [7]:
data3 = response_API3.json()
print(type(data3))

<class 'dict'>


In [20]:
institution_df = json_normalize(data['data'])
#print(institution_df.head())
#institution_df.to_csv('inst.csv')

institution_df2 = json_normalize(data2['data'])
#print(institution_df2.head())
#institution_df2.to_csv('inst2.csv')

institution_df3 = json_normalize(data3['data'])
#print(institution_df3.head())
#institution_df3.to_csv('inst3.csv')

   score data.ZIP  data.SASSER data.CHRTAGNT data.CONSERVE data.REGAGENT2  \
0      1    04073            0         STATE             N                  
1      1    72712            0         STATE             N                  
2      1    26003            0           OCC             N                  
3      1    24986            0         STATE             N                  
4      1    53946            0         STATE             N                  

     data.STNAME data.INSDATE data.TE06N528 data.TE06N529  ...  data.CHANGEC3  \
0          Maine   01/01/1934                              ...            NaN   
1       Arkansas   01/01/1934                              ...            NaN   
2  West Virginia   01/01/1934                              ...            NaN   
3  West Virginia   01/01/1934                              ...            NaN   
4      Wisconsin   01/01/1934                              ...            NaN   

  data.CHANGEC4 data.PRIORNAME6 data.PRIORNAME7 da

In [9]:
institution_df = institution_df.reindex(sorted(institution_df.columns), axis=1)
institution_df2 = institution_df2.reindex(sorted(institution_df2.columns), axis=1)
institution_df3 = institution_df3.reindex(sorted(institution_df3.columns), axis=1)

In [ ]:
#full_df = institution_df.append(institution_df.append(institution_df3))
#.append is being deprecated - use .concat instead

In [21]:
full_data = [institution_df, institution_df2, institution_df3]
df = pd.concat(full_data)
df.columns = df.columns.str.lstrip("data.")
#df.to_csv('inst_full.csv')
print(df.shape)

(27816, 168)


2. HERE ARE THREE STATISTICS ABOUT THIS DATA

In [23]:
bank_count = len(df)
print("There are", bank_count, "institutuions with data on the FDIC's API.\n")

active = df['ACTIVE'].value_counts()[0]
print("There are", active, "active institutions registered with the FDIC.\n")

inactive = df['ACTIVE'].value_counts()[1]
print("There are", inactive, "inactive institutions registered with the FDIC.\n")

There are 27816 institutuions with data on the FDIC's API.

There are 23098 active institutions registered with the FDIC.

There are 4718 inactive institutions registered with the FDIC.



In [31]:
active_df = df[df.ACTIVE == 0]
print(len(active_df))
print(active_df.head())

23098
   score    ZIP  SASSER CHRTAGNT CONSERVE REGAGENT2         STNAME  \
0      1  04073       0    STATE        N                    Maine   
1      1  72712       0    STATE        N                 Arkansas   
2      1  26003       0      OCC        N            West Virginia   
3      1  24986       0    STATE        N            West Virginia   
5      1  54306       0    STATE        N                Wisconsin   

      INSDATE TE06N528 TE06N529  ...  PRIORNAME8 PRIORNAME9 CHANGEC11  \
0  01/01/1934                    ...         NaN        NaN       NaN   
1  01/01/1934                    ...         NaN        NaN       NaN   
2  01/01/1934                    ...         NaN        NaN       NaN   
3  01/01/1934                    ...         NaN        NaN       NaN   
5  01/01/1934                    ...         NaN        NaN       NaN   

  CHANGEC10 INSDROPDATE count CHANGEC13  CHANGEC12 CHANGEC15 CHANGEC14  
0       NaN         NaN   NaN       NaN        NaN       NaN 

In [33]:
min = active_df["ASSET"].min()
max = active_df["ASSET"].max()
mean = active_df["ASSET"].mean()

print("The minimum assets held by an FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an FDIC registered institution are ${} \n".format(max))
print("The average assets held by an FDIC registered institution are ${} \n".format(mean))

The minimum assets held by an FDIC registered institution are $1.0 

The maximum assets held by an FDIC registered institution are $510083000.0 

The average assets held by an FDIC registered institution are $787717.1449738219 



2. HERE ARE THREE MORE STATISTICS ABOUT THIS DATA

In [34]:
min = df["ASSET"].min()
max = df["ASSET"].max()
mean = df["ASSET"].mean().round(2)

print("The minimum assets held by an FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an FDIC registered institution are ${} \n".format(max))
print("The average assets held by an FDIC registered institution are ${} \n".format(mean))

The minimum assets held by an FDIC registered institution are $1.0 

The maximum assets held by an FDIC registered institution are $3308575000.0 

The average assets held by an FDIC registered institution are $1618986.53 



In [ ]:
"""

Standarad deviation for later analysis

std = df["data.ASSET"].std().round(2)
top_std = (mean + std).round(2)
btm_std = (mean - std).round(2)

print(std)
print("Most FDIC registed banks have between ${} and ${} \n".format(btm_std, top_std))

"""

In [ ]:
# to format as currency
#.style.format('${0:,.2f}')

3. HERE IS A QUERY

In [ ]:
kc1_df = df.query("`data.ASSET` >= 610000 and `data.ASSET` <= 2610000")

4. HERE ARE THE SECOND AND THIRD COLUMNS OF THE NEW DF

In [ ]:
print(kc1_df.iloc[:, [1, 2]])

5. HERE ARE THE FIRST FOUR ROWS OF THE NEW DF

In [ ]:
print(kc1_df.head(4))

In [ ]:
#test = df.rename(columns=str.removeprefix("data."), index=str.title)
#print(test.head())

In [ ]:
"""

To add banks with FDIC violations

absolute_path = os.path.abspath('')
relative_path = "ots-enforcement-order-listing.csv"
full_path = os.path.join(absolute_path, relative_path)
enforcment_df = pd.read_csv(full_path)
print(enforcment_df.head())
"""

In [ ]:
"""

To remove unecessary columns

new_enf_df = enforcment_df

if 'DOCASSOCIATION' in new_enf_df.columns: 
    new_enf_df.drop(new_enf_df.iloc[:, 3:9], inplace=True, axis=1)

print(new_enf_df.head())
"""

In [ ]:
#drop o-v
#drop cj-dd
#